In [ ]:
import pandas as pd

In [11]:
# data import

import boto3
from io import BytesIO
import zipfile
from dotenv import load_dotenv
load_dotenv()
import os


aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
verbose=True
bucket_name="disease-risk-app"
zip_key="disease-risk-app_zip.zip"

s3 = boto3.client(
's3',
aws_access_key_id=aws_access_key_id,
aws_secret_access_key=aws_secret_access_key
)
response = s3.get_object(Bucket=bucket_name, Key=zip_key)
zip_bytes = BytesIO(response['Body'].read())
with zipfile.ZipFile(zip_bytes) as z:
    if verbose:
        print("Files in ZIP:", z.namelist())
    with z.open(z.namelist()[0]) as f:
        df = pd.read_csv(f)
# df.shape
# df.head(10)

Files in ZIP: ['disease-risk-app.csv']


In [ ]:
# # value counts

# cat_cols = df.select_dtypes(include='object').columns

# for col in cat_cols:
#     print(f"\n Value counts for column: '{col}'")
#     print(df[col].value_counts())


 Value counts for column: 'gender'
gender
Male      50132
Female    49868
Name: count, dtype: int64


In [ ]:
# drop use less columns
df= df.drop(columns="id")

# percentage of blank columns

total_rows = len(df)
blank_counts = df.isna().sum() + (df.astype(str).apply(lambda x: x.str.strip()) == '').sum()
blank_percentage = (blank_counts / total_rows) * 100
# print(blank_percentage)

In [14]:
df_filled = df.copy()


# Fill numeric columns with mean
num_cols = df_filled.select_dtypes(include='number').columns
for col in num_cols:
    df_filled[col] = df_filled[col].fillna(df_filled[col].mean())

# Fill categorical columns with mode
cat_cols = df_filled.select_dtypes(include='object').columns
for col in cat_cols:
    df_filled[col] = df_filled[col].fillna(df_filled[col].mode()[0])
# df_filled.head(10)

In [15]:
# removing outliers

from scipy.stats import zscore

# Select numeric columns
numeric_cols = df.select_dtypes(exclude='object').columns

# Compute Z-scores
z_scores = df[numeric_cols].apply(zscore)

# Create mask for rows where all Z-scores are within threshold
threshold = 3
mask = (z_scores.abs() < threshold).all(axis=1)
df_filtered = df[mask]
# print(df_filtered.shape)

In [21]:
import pandas as pd
import numpy as np

df_engg = df_filtered.copy()

# BMI category
df_engg['bmi_category'] = pd.cut(df_engg['bmi'], bins=[0, 18.5, 24.9, 29.9, np.inf],
                            labels=['Underweight', 'Normal', 'Overweight', 'Obese'])

# Blood pressure category (systolic only for simplicity)
df_engg['bp_category'] = pd.cut(df_engg['systolic_bp'], bins=[0, 120, 129, 139, 180, np.inf],
                            labels=['Normal', 'Elevated', 'Stage1', 'Stage2', 'Crisis'])

# Activity-to-sleep ratio
df_engg['activity_sleep_ratio'] = df_engg['daily_steps'] / (df_engg['sleep_hours'] + 1e-3)

# Hydration index
df_engg['hydration_index'] = df_engg['water_intake_l'] / (df_engg['calories_consumed'] + 1e-3)

# Interaction terms
df_engg['age_cholesterol'] = df_engg['age'] * df_engg['cholesterol']
df_engg['smoker_alcohol'] = df_engg['smoker'] * df_engg['alcohol']
df_engg['resting_bp_product'] = df_engg['resting_hr'] * df_engg['systolic_bp']

df_engg.head(30)

,age,gender,bmi,daily_steps,sleep_hours,water_intake_l,calories_consumed,smoker,alcohol,resting_hr,...,cholesterol,family_history,disease_risk,bmi_category,bp_category,activity_sleep_ratio,hydration_index,age_cholesterol,smoker_alcohol,resting_bp_product
0,56,Male,20.5,4198,3.9,3.4,1602,0,0,97,...,240,0,0,Normal,Stage2,1076.134325,0.002122,13440,0,15617
1,69,Female,33.3,14359,9.0,4.7,2346,0,1,68,...,207,0,0,Obese,Normal,1595.267193,0.002003,14283,0,7888
2,46,Male,31.6,1817,6.6,4.2,1643,0,1,90,...,296,0,0,Obese,Elevated,275.261324,0.002556,13616,0,11070
3,32,Female,38.2,15772,3.6,2.0,2460,0,0,71,...,175,0,0,Obese,Stage2,4379.894474,0.000813,5600,0,11715
4,60,Female,33.6,6037,3.8,4.0,3756,0,1,98,...,294,0,0,Obese,Stage1,1588.266246,0.001065,17640,0,13622
5,25,Male,27.3,19495,5.0,4.4,1301,0,1,73,...,284,0,0,Overweight,Normal,3898.220356,0.003382,7100,0,7811
6,78,Female,37.1,16739,9.5,4.2,3478,1,0,90,...,201,0,0,Obese,Normal,1761.814546,0.001208,15678,0,9900
7,38,Female,18.9,1726,4.8,1.7,3212,0,1,64,...,197,0,0,Normal,Normal,359.508436,0.000529,7486,0,7232
8,56,Female,18.2,1764,5.1,1.5,3740,0,1,91,...,237,1,0,Underweight,Normal,345.814546,0.000401,13272,0,10192
9,75,Male,23.5,9730,4.5,0.7,3571,0,1,54,...,157,1,0,Normal,Stage2,2161.741835,0.000196,11775,0,9558


In [25]:
# data encoding 
from sklearn.preprocessing import LabelEncoder


df_encoded = df_engg.copy()
ordinal_map= {"bmi_category":["Underweight","Normal","Overweight","Obese"],
              "bp_category":['Normal', 'Elevated', 'Stage1', 'Stage2', 'Crisis']}
cardinality_threshold=4

for col in df_encoded.select_dtypes(include=['object','category']).columns:
    unique_vals = df_encoded[col].nunique()

    # Ordinal encoding if ordinality exists
    if ordinal_map and col in ordinal_map:
        ordered_categories = ordinal_map[col]
        df_encoded[col] = df_encoded[col].astype(
            pd.CategoricalDtype(categories=ordered_categories, ordered=True)
        ).cat.codes

    # One-hot encoding for low-cardinality nominal features
    elif unique_vals <= cardinality_threshold:
        df_encoded = pd.get_dummies(df_encoded, columns=[col], drop_first=True)

    # Label encoding for high-cardinality nominal features
    else:
        le = LabelEncoder()
        df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))

bool_cols = df_encoded.select_dtypes(include='bool').columns
df_encoded[bool_cols] = df_encoded[bool_cols].astype(int)
df_encoded.head(20)
# bool_cols

,age,bmi,daily_steps,sleep_hours,water_intake_l,calories_consumed,smoker,alcohol,resting_hr,systolic_bp,...,family_history,disease_risk,bmi_category,bp_category,activity_sleep_ratio,hydration_index,age_cholesterol,smoker_alcohol,resting_bp_product,gender_Male
0,56,20.5,4198,3.9,3.4,1602,0,0,97,161,...,0,0,1,3,1076.134325,0.002122,13440,0,15617,1
1,69,33.3,14359,9.0,4.7,2346,0,1,68,116,...,0,0,3,0,1595.267193,0.002003,14283,0,7888,0
2,46,31.6,1817,6.6,4.2,1643,0,1,90,123,...,0,0,3,1,275.261324,0.002556,13616,0,11070,1
3,32,38.2,15772,3.6,2.0,2460,0,0,71,165,...,0,0,3,3,4379.894474,0.000813,5600,0,11715,0
4,60,33.6,6037,3.8,4.0,3756,0,1,98,139,...,0,0,3,2,1588.266246,0.001065,17640,0,13622,0
5,25,27.3,19495,5.0,4.4,1301,0,1,73,107,...,0,0,2,0,3898.220356,0.003382,7100,0,7811,1
6,78,37.1,16739,9.5,4.2,3478,1,0,90,110,...,0,0,3,0,1761.814546,0.001208,15678,0,9900,0
7,38,18.9,1726,4.8,1.7,3212,0,1,64,113,...,0,0,1,0,359.508436,0.000529,7486,0,7232,0
8,56,18.2,1764,5.1,1.5,3740,0,1,91,112,...,1,0,0,0,345.814546,0.000401,13272,0,10192,0
9,75,23.5,9730,4.5,0.7,3571,0,1,54,177,...,1,0,1,3,2161.741835,0.000196,11775,0,9558,1


In [26]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

target_column="disease_risk"
y=df_encoded[target_column]
x= df_encoded.drop(columns=target_column)
x_scaled= pd.DataFrame(scaler.fit_transform(x),columns=x.columns, index=x.index)
# x_scaled
# y
# print(y.value_counts())


In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
x_scaled, y,stratify=y ,test_size=0.2, random_state=42)

In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

classifiers = {
    'LogisticRegression': LogisticRegression(),
    # 'DecisionTree': DecisionTreeClassifier(),
    # 'RandomForest': RandomForestClassifier(),
    # 'GradientBoosting': GradientBoostingClassifier(),
    # 'XGBoost': XGBClassifier(verbosity=0),
    # 'CatBoost': CatBoostClassifier(verbose=0),
    # 'LightGBM': LGBMClassifier(),
    # 'KNN': KNeighborsClassifier(),
    # 'NaiveBayes': GaussianNB(),
    # 'LDA': LinearDiscriminantAnalysis()
}

In [44]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,roc_auc_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek

import mlflow
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/arpitmishra917/disease-risk-app.mlflow')
dagshub.init(repo_owner='arpitmishra917', repo_name='disease-risk-app', mlflow=True)


random_state=42
resamplers = {
    'SMOTE': SMOTE(random_state=random_state),
    # 'RandomUnderSampler': RandomUnderSampler(random_state=random_state),
    # 'SMOTE_Tomek': SMOTETomek(random_state=random_state)
}

all_results = {}
for resample_name, resampler in resamplers.items():
    X_resampled, y_resampled = resampler.fit_resample(X_train, y_train)
    print(f"\n🔁 Resampling: {resample_name} — Resampled shape: {X_resampled.shape}")


    for name, model in classifiers.items():
        with mlflow.start_run(run_name=f"{resample_name}_{name}"):
            model.fit(X_resampled, y_resampled)
            y_pred = model.predict(X_test)
            y_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None

            thresh = 0.485
            y_pred_thresh = (y_proba >= thresh).astype(int)

            result= {
            'Accuracy': accuracy_score(y_test, y_pred_thresh),
            'Precision': precision_score(y_test, y_pred_thresh),
            'Recall': recall_score(y_test, y_pred_thresh),
            'F1 Score': f1_score(y_test, y_pred_thresh),
            'ROC AUC': roc_auc_score(y_test, y_proba) if y_proba is not None else None
            # 'Confusion Matrix': confusion_matrix(y_test, y_pred)
        }
            
            all_results[(resample_name, name)] = result

            for metric_name, metric_value in result.items():
                mlflow.log_metric(metric_name, metric_value)


all_results



Accessing as arpitmishra917

Initialized MLflow to track repo "arpitmishra917/disease-risk-app"

Repository arpitmishra917/disease-risk-app initialized!


🔁 Resampling: SMOTE — Resampled shape: (120286, 21)
🏃 View run SMOTE_LogisticRegression at: https://dagshub.com/arpitmishra917/disease-risk-app.mlflow/#/experiments/0/runs/391b40b1e66346ebbfabf147b59f78b2
🧪 View experiment at: https://dagshub.com/arpitmishra917/disease-risk-app.mlflow/#/experiments/0


{('SMOTE', 'LogisticRegression'): {'Accuracy': 0.3132,
  'Precision': 0.2492854693037613,
  'Recall': 0.878525382755842,
  'F1 Score': 0.38836940065900794,
  'ROC AUC': 0.503890032710107}}

In [37]:
# Extract and sort importances
importances = model.feature_importances_
feature_names = x_scaled.columns
sorted_features = sorted(zip(feature_names, importances), key=lambda x: x[1], reverse=True)

# Display top features
for feature, score in sorted_features[:10]:
    print(f"{feature}: {score:.4f}")

systolic_bp: 0.0769
water_intake_l: 0.0753
diastolic_bp: 0.0753
resting_hr: 0.0706
bmi: 0.0684
cholesterol: 0.0633
sleep_hours: 0.0620
calories_consumed: 0.0595
age: 0.0590
daily_steps: 0.0587
